# Real-Time Face Recognition : 
## Phases : 
1. Face Detection and Data Gathering
2. Train the Recognizer
3. Face Recognition



## Creating faces with ids

In [1]:
import cv2
import os
import time
import datetime
import numpy as np
from PIL import Image


### Creating faces using video recording

In [ ]:
# Creating faces using video recording
cam = cv2.VideoCapture(0)
cam.set(3, 1200) # set video width
cam.set(4, 800) # set video height

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# For each person, enter one numeric face id
face_id = input('\n enter user id end press <return> ==>  ')

print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0

while True:
    ret, img = cam.read()
    # img = cv2.flip(img, -1) # flip video image vertically
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1

        # Save the captured image into the datasets folder
        cv2.imwrite("dataset/users." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

        cv2.imshow('image', img)

    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video

    time.sleep(0.2)
    if k == 27:
        break
    elif count >= 200: # Take 30 face sample and stop video
         break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

### Creating faces using images

In [ ]:
# Creating faces using images
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')

root = "/home/vipul/Vipul/Learning Technology/ML-AI Vipul/Face Recognition/face_recognition2/"
raw_data_folder_path = "raw_data"
user_id_starting = 1

# Function to get list of image files from directory
def get_image_files(directory):
    face_user_name_list = [] 
    user_id = user_id_starting
    for path in os.listdir(directory):
        face_user_name_list.append(path)
        
        folder_path = os.path.join(raw_data_folder_path, path)

        count = 0
        for file in os.listdir(folder_path):
            try:
                image_file_path = os.path.join(folder_path, file)
                img = cv2.imread(image_file_path)
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_detector.detectMultiScale(gray, 1.4, 6)
    
                for (x, y, w, h) in faces:
                    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    count += 1
    
                    roi_gray = gray[y:y+h, x:x+w]
                    roi_color = img[y:y+h, x:x+w]
        
                    eyes = eye_classifier.detectMultiScale(roi_gray)
                    eye_count = 0
                    for (ex,ey,ew,eh) in eyes:
                        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)
                        cv2.imshow('image', img)
                        eye_count += 1
    
                        if eye_count == 2:
                            # Save the captured image into the datasets folder
                            cv2.imwrite(f"dataset/users.{user_id}.{count}.jpg", gray[y:y+h, x:x+w])
        
                    count += 1
            except Exception as e:
                print(e)
                continue
    
            k = cv2.waitKey(100) & 0xff  # Press 'ESC' for exiting video
    
            if k == 27 or count >= 500:  # Take 200 face samples or press ESC to exit
                break

        user_id += 1

    print("USER NAME LIST : ", face_user_name_list)

a = get_image_files(directory="raw_data")


## Recognizer Training

In [ ]:
# Path for face image database
path = 'dataset'
recognizer = cv2.face.LBPHFaceRecognizer.create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        try:
            PIL_img = Image.open(imagePath).convert('L') # grayscale
            img_numpy = np.array(PIL_img,'uint8')
            id = int(os.path.split(imagePath)[-1].split(".")[1])
            faces = detector.detectMultiScale(img_numpy)
            for (x,y,w,h) in faces:
                faceSamples.append(img_numpy[y:y+h,x:x+w])
                ids.append(id)
        except Exception as e:
            print(e)
    return faceSamples,ids
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write('trainer/trainer.yml') 
# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

### Model(Recognizer) Testing

In [2]:
# names related to ids: example ==> Marcelo: id=1,  etc
names = ['None', 'hansika', 'anushka', 'samantha', 'dipika', 'shraddha']

def real_time_face_recognition():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/trainer.yml')
    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath)
    font = cv2.FONT_HERSHEY_SIMPLEX
    #iniciate id counter
    id = 0
    
    # Initialize and start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 1200) # set video width
    cam.set(4, 800) # set video height
    # Define min window size to be recognized as a face
    minW = 0.1*cam.get(3)
    minH = 0.1*cam.get(4)

    while True:
        ret, img = cam.read()
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        faces = faceCascade.detectMultiScale( 
            gray,
            scaleFactor = 1.4,
            minNeighbors = 6,
            minSize = (int(minW), int(minH)),
           )
        
        for(x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
            
            # If confidence is less than 100 ==> "0" : perfect match 
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
                
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
            cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
            cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
        
        cv2.imshow('camera',img) 
        k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
        if k == 27:
            break
            
    # Release the webcam
    cam.release()
    cv2.destroyAllWindows()

def image_face_recognition(image_path):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/trainer.yml')
    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath)
    font = cv2.FONT_HERSHEY_SIMPLEX

    try:
        # Load the image
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(30, 30))
        
        for (x, y, w, h) in faces:
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
            print(id, confidence)
            if (confidence < 100 and (100 - confidence) > 20):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))
                
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(img, str(id), (x+5, y-5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, str(confidence), (x+5, y+h-5), font, 1, (255, 255, 0), 1)

        img = cv2.resize(img, (1500, 900))    
        cv2.imshow('image', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    except Exception as e:
        print(e)

In [ ]:
# Main menu
while True:
    print("Select an option:")
    print("1. Real-time Face Recognition (Webcam)")
    print("2. Image Face Recognition")
    print("3. Exit")
    choice = input("Enter your choice (1, 2, or 3): ")

    if choice == '1':
        real_time_face_recognition()
    elif choice == '2':
        image_path = "testing_data/test-images/test-image-5.jpg"

        if os.path.exists(image_path):
            image_face_recognition(image_path)
        else:
            print("Invalid image path. Please try again.")
        break
    elif choice == '3':
        break
    else:
        print("Invalid choice. Please enter 1, 2, or 3.")

Select an option:
1. Real-time Face Recognition (Webcam)
2. Image Face Recognition
3. Exit


Enter your choice (1, 2, or 3):  2


2 59.142502503835274
3 52.36904887419371
5 50.461781862716286
3 45.22669662740299
3 85.29684588938082
3 83.13428168375714
4 41.51178293480041
2 35.40283474256552
3 38.44253634398156
4 31.102144141144997
3 97.96958365855295
4 50.82224857218714
4 41.855134658424085
5 67.68629047859257
3 41.562693831641525
4 11.885151217173698
4 52.050089144111624
